<a href="https://colab.research.google.com/github/Jungseunggi/U-Net_Review/blob/main/U_Net_tesorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from tensorflow.keras.layers import Dense, Input, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization,Add,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LeakyReLU, ReLU, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, UpSampling2D, concatenate, Cropping1D, Cropping2D, Cropping3D, CenterCrop
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K

<img src='https://user-images.githubusercontent.com/102225200/201018530-6183dc77-cf78-4d7f-8a67-7f39529df194.png' width=800>

## 2차 

In [8]:
# 임의 사이즈대로 설정
def unet(input_size = (512,512,1)):
  input = Input(input_size)
  # 논문에 의하면 패딩은 없음으로 되어있지만 패링대신 미러링으로 크기를 맞춰주기 때문에 same, 활성함수는 relu 마지막 레이어만 softmax
  padding = 'same'
  activation = 'relu'
  
# Contracting path

  # 다른 참고 코드를 보면 패딩을 다 same으로 사용하던데 사용하기 편해서 그런 것 같음
  conv = Conv2D(64, 3, activation = activation, padding = padding)(input)
  conv = Conv2D(64, 3, activation = activation, padding = padding)(conv)
  # 패딩을 same으로 썻기 떄문에 dropout 으로 crop을 함
  crop1 = Dropout(0.5)(conv)
  pool = MaxPooling2D(pool_size=(2, 2))(conv)

  conv = Conv2D(128, 3, activation = activation, padding = padding)(pool)
  conv = Conv2D(128, 3, activation = activation, padding = padding)(conv)
  crop2 = Dropout(0.5)(conv)
  pool = MaxPooling2D(pool_size=(2, 2))(conv)

  conv = Conv2D(256, 3, activation = activation, padding = padding)(pool)
  conv = Conv2D(256, 3, activation = activation, padding = padding)(conv)
  crop3 = Dropout(0.5)(conv)
  pool = MaxPooling2D(pool_size=(2, 2))(conv)

  conv = Conv2D(512, 3, activation = activation, padding = padding)(pool)
  conv = Conv2D(512, 3, activation = activation, padding = padding)(conv)
  crop4 = Dropout(0.5)(conv)
  pool = MaxPooling2D(pool_size=(2, 2))(conv)

  conv = Conv2D(1024, 3, activation = activation, padding = padding)(pool)
  conv = Conv2D(1024, 3, activation = activation, padding = padding)(conv)

# Expanding path

  # UpSampling2D만 사용하면 그냥 해상도만 뻥튀기시켜주므로 그 전에 Conv2Dtranspose로 먼저 특징을 잡아줌
  conv = Conv2DTranspose(512, 3, padding=padding)(conv)
  conv = UpSampling2D(size=(2,2))(conv)
  conv = concatenate([crop4,conv])
  conv = Conv2D(512, 3, activation = activation, padding = padding)(conv)
  conv = Conv2D(512, 3, activation = activation, padding = padding)(conv)

  conv = Conv2DTranspose(256, 3, padding="same")(conv)
  conv = UpSampling2D(size=(2,2))(conv)
  conv = concatenate([crop3,conv])
  conv = Conv2D(256, 3, activation = activation, padding = padding)(conv)
  conv = Conv2D(256, 3, activation = activation, padding = padding)(conv)

  conv = Conv2DTranspose(128, 3, padding="same")(conv)
  conv = UpSampling2D(size=(2,2))(conv)
  conv = concatenate([crop2,conv])
  conv = Conv2D(128, 3, activation = activation, padding = padding)(conv)
  conv = Conv2D(128, 3, activation = activation, padding = padding)(conv)

  conv = Conv2DTranspose(64, 3, padding="same")(conv)
  conv = UpSampling2D(size=(2,2))(conv)
  conv = concatenate([crop1,conv])
  conv = Conv2D(64, 3, activation = activation, padding = padding)(conv)
  conv = Conv2D(64, 3, activation = activation, padding = padding)(conv)

  # # 최종레이어
  conv = Conv2D(2, 1, activation="softmax", padding = padding)(conv)
  
  
  
  model = Model(inputs = input, outputs=[conv])

  return model

model = unet()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_53 (Conv2D)             (None, 512, 512, 64  640         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_54 (Conv2D)             (None, 512, 512, 64  36928       ['conv2d_53[0][0]']              
                                )                                                           

## 1차

In [ ]:
# 논문에 나온 사이즈대로 설정
def unet(input_size = (572,572,1)):
  input = Input(input_size)
  # 논문에 의하면 패딩은 없음으로 valid, 활성함수는 relu 마지막 레이어만 softmax
  padding = 'valid'
  activation = 'relu'
  
# Contracting path

  # 다른 참고 코드를 보면 패딩을 다 same으로 사용하던데 사용하기 편해서 그런 것 같음
  conv = Conv2D(64, 3, activation = activation, padding = padding)(input)
  conv = Conv2D(64, 3, activation = activation, padding = padding)(conv)
  # dropout 보단 크기를 맞춰 자르는거에 초점을 맞춰서 centercrop으로 적용하는 것이 맞다고 생각함
  crop1 = CenterCrop(392,392)(conv)
  pool = MaxPooling2D(pool_size=(2, 2))(conv)

  conv = Conv2D(128, 3, activation = activation, padding = padding)(pool)
  conv = Conv2D(128, 3, activation = activation, padding = padding)(conv)
  crop2 = CenterCrop(200,200)(conv)
  pool = MaxPooling2D(pool_size=(2, 2))(conv)

  conv = Conv2D(256, 3, activation = activation, padding = padding)(pool)
  conv = Conv2D(256, 3, activation = activation, padding = padding)(conv)
  crop3 = CenterCrop(104,104)(conv)
  pool = MaxPooling2D(pool_size=(2, 2))(conv)

  conv = Conv2D(512, 3, activation = activation, padding = padding)(pool)
  conv = Conv2D(512, 3, activation = activation, padding = padding)(conv)
  crop4 = CenterCrop(56,56)(conv)
  pool = MaxPooling2D(pool_size=(2, 2))(conv)

  conv = Conv2D(1024, 3, activation = activation, padding = padding)(pool)
  conv = Conv2D(1024, 3, activation = activation, padding = padding)(conv)

# Expanding path

  # UpSampling2D만 사용하면 그냥 해상도만 뻥튀기시켜주므로 그 전에 Conv2Dtranspose로 먼저 특징을 잡아줌
  conv = Conv2DTranspose(512, 3, padding="same")(conv)
  conv = UpSampling2D(size=(2,2))(conv)
  conv = concatenate([crop4,conv])
  conv = Conv2D(512, 3, activation = activation, padding = padding)(conv)
  conv = Conv2D(512, 3, activation = activation, padding = padding)(conv)

  conv = Conv2DTranspose(256, 3, padding="same")(conv)
  conv = UpSampling2D(size=(2,2))(conv)
  conv = concatenate([crop3,conv])
  conv = Conv2D(256, 3, activation = activation, padding = padding)(conv)
  conv = Conv2D(256, 3, activation = activation, padding = padding)(conv)

  conv = Conv2DTranspose(128, 3, padding="same")(conv)
  conv = UpSampling2D(size=(2,2))(conv)
  conv = concatenate([crop2,conv])
  conv = Conv2D(128, 3, activation = activation, padding = padding)(conv)
  conv = Conv2D(128, 3, activation = activation, padding = padding)(conv)

  conv = Conv2DTranspose(64, 3, padding="same")(conv)
  conv = UpSampling2D(size=(2,2))(conv)
  conv = concatenate([crop1,conv])
  conv = Conv2D(64, 3, activation = activation, padding = padding)(conv)
  conv = Conv2D(64, 3, activation = activation, padding = padding)(conv)

  # 최종레이어
  conv = Conv2D(2, 1, activation="softmax", padding = padding)(conv)
  
  
  
  model = Model(inputs = input, outputs=[conv])

  return model

model = unet()
model.summary()

Model: "model_37"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_56 (InputLayer)          [(None, 572, 572, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_655 (Conv2D)            (None, 570, 570, 64  640         ['input_56[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_656 (Conv2D)            (None, 568, 568, 64  36928       ['conv2d_655[0][0]']             
                                )                                                          